#Decision tree classifier to find the best parameters and performance


In [7]:
# Data preparation

import pandas as pd
from sklearn import preprocessing
from IPython.display import display, HTML

df_loan200K=pd.read_csv('Data_Loans_200K.csv', header=0)
df_loan200K = df_loan200K.drop('Unnamed: 0', axis=1)

print(df_loan200K.shape)
cols_loan200K = df_loan200K.columns
 
    
print('Column Name, DataTypes, MissingValues in Loan 200K CSV\n')
for i in cols_loan200K:
    print(i,',', df_loan200K[i].dtype , ',', df_loan200K[i].isnull().any())

##If Yes, fill in missing values by mean values or most frequent nominal values.

df_loan200K["Credit Score"].fillna(df_loan200K["Credit Score"].mean(), inplace=True)
df_loan200K["Annual Income"].fillna(df_loan200K["Annual Income"].mean(), inplace=True)
df_loan200K["Years in current job"].fillna(df_loan200K["Years in current job"].mode().iloc[0], inplace=True)
df_loan200K["Months since last delinquent"].fillna(df_loan200K["Months since last delinquent"].mean(), inplace=True)
df_loan200K["Maximum Open Credit"].fillna(df_loan200K["Maximum Open Credit"].mean(), inplace=True)
df_loan200K["Bankruptcies"].fillna(df_loan200K["Bankruptcies"].mean(), inplace=True)
df_loan200K["Tax Liens"].fillna(df_loan200K["Tax Liens"].mean(), inplace=True)

print('\nColumn Name, DataTypes, MissingValues after filling with Mean and Frequent repeated nominal value in Loan 200K CSV\n')
for i in cols_loan200K:
    print(i,',', df_loan200K[i].dtype , ',', df_loan200K[i].isnull().any()) 
   
# encode labels
y = df_loan200K['Term'] # define label as nominal values
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y) # encode nominal labels to integers #####################################
df_loan200K['Term'] = y_encoded

# Data preprocessing ################################################################################
print('Column Datatypes:\n',df_loan200K.dtypes)

# convert all nominal variables to binary variables
df_num=df_loan200K.copy(deep=True) 
# create new binary columns
df_dummies=pd.get_dummies(df_num[['Loan Status','Years in current job','Home Ownership','Purpose']])
# add them to dataframe
df_num=df_num.join(df_dummies)
# drop original columns
df_num=df_num.drop('Loan Status',axis=1)
df_num=df_num.drop('Years in current job',axis=1)
df_num=df_num.drop('Home Ownership', axis=1)
df_num=df_num.drop('Purpose', axis=1)

# drop extra binary columns, since we only need N-1 binary columns
df_num=df_num.drop('Loan Status_Charged Off', axis=1)
df_num=df_num.drop('Years in current job_10+ years', axis=1)
df_num=df_num.drop('Home Ownership_Own Home', axis=1)
df_num=df_num.drop('Purpose_wedding', axis=1)

# print out and display dataframe as table in HTML
display(HTML(df_num.head(10).to_html()))

(200000, 17)
Column Name, DataTypes, MissingValues in Loan 200K CSV

Loan Status , object , False
Current Loan Amount , int64 , False
Term , object , False
Credit Score , float64 , True
Annual Income , float64 , True
Years in current job , object , True
Home Ownership , object , False
Purpose , object , False
Monthly Debt , float64 , False
Years of Credit History , float64 , False
Months since last delinquent , float64 , True
Number of Open Accounts , int64 , False
Number of Credit Problems , int64 , False
Current Credit Balance , int64 , False
Maximum Open Credit , float64 , True
Bankruptcies , float64 , True
Tax Liens , float64 , True

Column Name, DataTypes, MissingValues after filling with Mean and Frequent repeated nominal value in Loan 200K CSV

Loan Status , object , False
Current Loan Amount , int64 , False
Term , object , False
Credit Score , float64 , False
Annual Income , float64 , False
Years in current job , object , False
Home Ownership , object , False
Purpose , object ,

In [8]:
# 02. DecisionTreeClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score

y = df_num['Term']
x = df_num.drop('Term', axis=1)

# by hold-out evaluation with Gini impurity criterion
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
clf=DecisionTreeClassifier(criterion='gini', max_depth=10, ccp_alpha = 0.1) # note: there are many parameters in API
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
rec=recall_score(y_test, y_pred, average='macro')
print('By hold-out evaluation: accuracy = ',acc, ', precison = ', pre, ', recall = ', rec)


By hold-out evaluation: accuracy =  0.61142 , precison =  0.30571 , recall =  0.5


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
# by hold-out evaluation with Shannon information gain criterion entropy
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
clf=DecisionTreeClassifier(criterion='entropy', max_depth=10, ccp_alpha = 0.1) # note: there are many parameters in API
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
rec=recall_score(y_test, y_pred, average='macro')
print('By hold-out evaluation: accuracy = ',acc, ', precison = ', pre, ', recall = ', rec)


By hold-out evaluation: accuracy =  0.61256 , precison =  0.30628 , recall =  0.5


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
# 03. Visualizing decision trees

from sklearn import tree
from sklearn.model_selection import ShuffleSplit
import graphviz

clf_entropy=DecisionTreeClassifier(criterion='entropy', max_depth=100, max_features='auto', ccp_alpha = 0.1)
cv = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
rst_entropy = cross_validate(clf_entropy, x, y, cv=cv, scoring='accuracy', return_estimator=True)
# get fitted trees for each fold
trees_entropy = rst_entropy['estimator']
# print accuracy scores for each fold
scores_entropy = rst_entropy['test_score']
print("Test score: ",scores_entropy)

# select the best model
clf_entropy = trees_entropy[1]

# encode labels
labels = le.inverse_transform([0,1])

print('features: ', x.columns.to_list())
print('labels: ', labels)

# DOT data
dot_data_entropy = tree.export_graphviz(clf_entropy, out_file=None, 
                                feature_names=x.columns.to_list(),  
                                class_names=labels,
                                filled=True)

# Draw graph
graph_entropy = graphviz.Source(dot_data_entropy, format="png") 
graph_entropy.render("MyDecisionTree_entropy_200k")
graph_entropy


Test score:  [0.61222 0.6104  0.61012 0.61372 0.61174]
features:  ['Current Loan Amount', 'Credit Score', 'Annual Income', 'Monthly Debt', 'Years of Credit History', 'Months since last delinquent', 'Number of Open Accounts', 'Number of Credit Problems', 'Current Credit Balance', 'Maximum Open Credit', 'Bankruptcies', 'Tax Liens', 'Loan Status_Fully Paid', 'Years in current job_1 year', 'Years in current job_2 years', 'Years in current job_3 years', 'Years in current job_4 years', 'Years in current job_5 years', 'Years in current job_6 years', 'Years in current job_7 years', 'Years in current job_8 years', 'Years in current job_9 years', 'Years in current job_< 1 year', 'Home Ownership_HaveMortgage', 'Home Ownership_Home Mortgage', 'Home Ownership_Rent', 'Purpose_Business Loan', 'Purpose_Buy House', 'Purpose_Buy a Car', 'Purpose_Debt Consolidation', 'Purpose_Educational Expenses', 'Purpose_Home Improvements', 'Purpose_Medical Bills', 'Purpose_Other', 'Purpose_Take a Trip', 'Purpose_majo

In [16]:
clf_gini=DecisionTreeClassifier(criterion='gini', max_depth=100, max_features='auto', ccp_alpha = 0.1)
cv = ShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
rst_gini = cross_validate(clf_gini, x, y, cv=cv, scoring='accuracy', return_estimator=True)
# get fitted trees for each fold
trees_gini = rst_gini['estimator']
# print accuracy scores for each fold
scores_gini = rst_gini['test_score']
print("Test score: ",scores_gini)

# select the best model
clf_gini = trees_gini[1]

# encode labels
labels = le.inverse_transform([0,1])

print('features: ', x.columns.to_list())
print('labels: ', labels)

# DOT data
dot_data_gini = tree.export_graphviz(clf_gini, out_file=None, 
                                feature_names=x.columns.to_list(),  
                                class_names=labels,
                                filled=True)

# Draw graph
graph_gini = graphviz.Source(dot_data_gini, format="png") 
graph_gini.render("MyDecisionTree_gini_200K")
graph_gini


Test score:  [0.61222 0.6104  0.61012 0.61372 0.61174]
features:  ['Current Loan Amount', 'Credit Score', 'Annual Income', 'Monthly Debt', 'Years of Credit History', 'Months since last delinquent', 'Number of Open Accounts', 'Number of Credit Problems', 'Current Credit Balance', 'Maximum Open Credit', 'Bankruptcies', 'Tax Liens', 'Loan Status_Fully Paid', 'Years in current job_1 year', 'Years in current job_2 years', 'Years in current job_3 years', 'Years in current job_4 years', 'Years in current job_5 years', 'Years in current job_6 years', 'Years in current job_7 years', 'Years in current job_8 years', 'Years in current job_9 years', 'Years in current job_< 1 year', 'Home Ownership_HaveMortgage', 'Home Ownership_Home Mortgage', 'Home Ownership_Rent', 'Purpose_Business Loan', 'Purpose_Buy House', 'Purpose_Buy a Car', 'Purpose_Debt Consolidation', 'Purpose_Educational Expenses', 'Purpose_Home Improvements', 'Purpose_Medical Bills', 'Purpose_Other', 'Purpose_Take a Trip', 'Purpose_majo